# 📦 Telco Customer - 이탈 예측 (Binary Classification)
🎯  통신 서비스를 이용하는 고객의 정보(계약 유형, 요금, 서비스 사용 내역 등)를 바탕으로 서비스 이탈 여부를 예측하는 이진 분류 문제

## ✅ 1. 데이터 전처리

### 📌 1-1. 데이터 로딩

In [1]:
import pandas as pd

# 데이터 불러오기
df = pd.read_csv("/Users/lee_hyejoo/Desktop/hyejoo/학교/3학년 1학기/머신러닝/중간_대체/고객_이탈_예측, Telco Customer Churn - Binary Classification/WA_Fn-UseC_-Telco-Customer-Churn.csv")

# 기본 정보 확인
display(df.head())          # 상위 5개 행 출력
print(df.info())            # 컬럼 수, 데이터 타입, 결측치 확인
print(df.isnull().sum())    # 결측치 수 확인

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


### 📌 1-2. 결측치 처리 및 TotalCharges 수치형 변환
- TotalCharges는 수치형인데도 불구하고 object인 이유는 공백 문자열이 섞여 있음
- pd.to_numeric(..., errors='coerce')로 숫자로 변환하고, 변환 실패한 값은 NaN으로 바꿔서 정리

In [3]:
# TotalCharges 공백 → NaN 처리
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# 변환 후 결측치 확인
print(" TotalCharges 결측치 수:", df['TotalCharges'].isna().sum())

# 결측치 행 제거
df = df.dropna(subset=['TotalCharges'])

# 타입 확인
print(" TotalCharges 타입:", df['TotalCharges'].dtype)

 TotalCharges 결측치 수: 0
 TotalCharges 타입: float64


### 📌 1-3. 범주형 변수 인코딩 (One-Hot Encoding)
- 머신러닝 모델에 사용하기 위해 문자형 변수들을 수치화
- 특히 다음 변수들은 계약 유형, 결제 방식 등 이탈에 큰 영향을 줄 수 있는 중요한 피처들

In [11]:
#  범주형 변수 확인
cat_cols = df.select_dtypes(include=["object"]).columns.tolist()
cat_cols.remove("Churn")  # 타깃 값은 제외

#  One-hot 인코딩 수행
df_encoded = pd.get_dummies(df, columns=cat_cols, drop_first=True)

#  타깃 변수 이진화
df_encoded["Churn"] = df_encoded["Churn"].map({"Yes": 1, "No": 0})

#  결과 확인
print("인코딩 후 shape:", df_encoded.shape)
df_encoded.head()

인코딩 후 shape: (7032, 31)


,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,Churn,gender_Male,Partner_Yes,Dependents_Yes,PhoneService_Yes,MultipleLines_No phone service,...,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0,1,29.85,29.85,0,False,True,False,False,True,...,False,False,False,False,False,False,True,False,True,False
1,0,34,56.95,1889.50,0,True,False,False,True,False,...,False,False,False,False,True,False,False,False,False,True
2,0,2,53.85,108.15,1,True,False,False,True,False,...,False,False,False,False,False,False,True,False,False,True
3,0,45,42.30,1840.75,0,True,False,False,False,True,...,False,False,False,False,True,False,False,False,False,False
4,0,2,70.70,151.65,1,False,False,False,True,False,...,False,False,False,False,False,False,True,False,True,False


### 📌 1-4. 스케일링 (월별 요금, 총 지출금액)
- 대부분의 모델들은 특성의 스케일 차이에 민감, 특히 MonthlyCharges, TotalCharges는 값의 범위가 커서 정규화(normalization) 필요

In [17]:
from sklearn.preprocessing import StandardScaler

#  복사본 생성
df_scaled = df_encoded.copy()

#  스케일링 대상 컬럼
scale_cols = ["MonthlyCharges", "TotalCharges"]

#  StandardScaler 적용
scaler = StandardScaler()
df_scaled[scale_cols] = scaler.fit_transform(df_scaled[scale_cols])

#  결과 확인
df_scaled[scale_cols].describe()


,MonthlyCharges,TotalCharges
count,7.032000e+03,7.032000e+03
mean,-1.247896e-16,5.582691e-17
std,1.000071e+00,1.000071e+00
min,-1.547283e+00,-9.990692e-01
25%,-9.709769e-01,-8.302488e-01
50%,1.845440e-01,-3.908151e-01
75%,8.331482e-01,6.668271e-01
max,1.793381e+00,2.824261e+00


---

## 🤖 2. 모델 학습 및 비교

### 📌 2-1. 기본 모델 - Logistic Regression

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

#  타겟과 피처 분리
X = df_scaled.drop("Churn", axis=1)
y = df_scaled["Churn"].astype(int)  # 'Yes', 'No' → 1, 0으로 변환됨

#  데이터 분할 (train: 80%, test: 20%)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

#  모델 정의 및 학습
lr_model = LogisticRegression(max_iter=1000, random_state=42)
lr_model.fit(X_train, y_train)

#  예측 및 평가
y_pred_lr = lr_model.predict(X_test)

print(" [Logistic Regression]")
print(classification_report(y_test, y_pred_lr, digits=4))


 [Logistic Regression]
              precision    recall  f1-score   support

           0     0.8512    0.8858    0.8681      1033
           1     0.6446    0.5722    0.6062       374

    accuracy                         0.8024      1407
   macro avg     0.7479    0.7290    0.7372      1407
weighted avg     0.7962    0.8024    0.7985      1407



🔎 해석

지표 | 의미 | 해석

Precision (Churn=1) | 예측한 이탈 고객 중 실제 이탈 비율 | 64.46% → 예측한 이탈 고객 중 약 64%만 진짜 이탈

Recall (Churn=1) | 실제 이탈 고객 중 모델이 맞춘 비율 | 57.22% → 실제 이탈 고객 중 약 57%만 탐지함

F1-Score (Churn=1) | Precision과 Recall의 조화 평균 | 60.62%

Accuracy | 전체 예측 중 맞춘 비율 | 80.24% → imbalance (불균형) 데이터에서는 해석 주의

➡️ 이 모델은 이탈 고객 예측을 꽤 잘 했지만, 아직 recall이 낮아 일부 이탈 고객을 놓치고 있음

➡️ 전체 정확도는 높지만, 이건 클래스 불균형 영향이 있으니 precision / recall을 중점적으로 평가하는 게 중요

### 📌 2-2. 고급 모델 – Random Forest

In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# 모델 정의 및 학습
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# 예측
y_pred_rf = rf_model.predict(X_test)

# 결과 확인
print(" [Random Forest]")
print(classification_report(y_test, y_pred_rf, digits=4))

 [Random Forest]
              precision    recall  f1-score   support

           0     0.8335    0.8867    0.8593      1033
           1     0.6201    0.5107    0.5601       374

    accuracy                         0.7868      1407
   macro avg     0.7268    0.6987    0.7097      1407
weighted avg     0.7768    0.7868    0.7798      1407



🔍 해석

이탈 고객(1) 예측에 대해:
- Precision 0.6201: 이탈이라고 예측한 것 중 약 62%만 실제 이탈
- Recall 0.5107: 실제 이탈 고객 중 약 51%만 올바르게 예측 
- ➡️ 이탈을 놓치는 경우가 많음 (Recall 낮음)

비이탈 고객(0) 예측 성능은 상대적으로 우수:
- Precision 0.8335, Recall 0.8867로 높은 편
- 전체적으로 Logistic Regression보다 recall, f1-score가 떨어지는 경향 (특히 클래스 1에서 성능 저하 있음)

### 📌 2-3. 고급 모델 – XGBoost

In [23]:
from xgboost import XGBClassifier

# 모델 정의 및 학습
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_model.fit(X_train, y_train)

# 예측
y_pred_xgb = xgb_model.predict(X_test)

# 결과 확인
print(" [XGBoost]")
print(classification_report(y_test, y_pred_xgb, digits=4))

 [XGBoost]
              precision    recall  f1-score   support

           0     0.8304    0.8577    0.8438      1033
           1     0.5676    0.5160    0.5406       374

    accuracy                         0.7669      1407
   macro avg     0.6990    0.6869    0.6922      1407
weighted avg     0.7605    0.7669    0.7632      1407



/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [11:04:41] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


🔍 해석
- 이탈 고객(1) 탐지에서 precision은 56.8%, recall은 51.6%로 낮은 편 → 예측된 이탈 중 절반만 진짜 이탈이고, 실제 이탈 중 절반 정도만 탐지됨 → 특히 Recall이 낮아 이탈 고객을 놓치는 경우가 많은 상황

- 비이탈 고객(0) 에서는 성능 준수 → Recall이 85.8%로, 잘 분류된 편
- 전체적으로 Random Forest와 유사하거나 약간 더 낮은 성능

### 📌 2-4. 고급 모델 – LightGBM

In [24]:
from lightgbm import LGBMClassifier

# 모델 정의 및 학습
lgb_model = LGBMClassifier(random_state=42)
lgb_model.fit(X_train, y_train)

# 예측
y_pred_lgb = lgb_model.predict(X_test)

# 결과 확인
print(" [LightGBM]")
print(classification_report(y_test, y_pred_lgb, digits=4))

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 1495, number of negative: 4130
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000591 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 637
[LightGBM] [Info] Number of data points in the train set: 5625, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.265778 -> initscore=-1.016151
[LightGBM] [Info] Start training from score -1.016151
 [LightGBM]
              precision    recall  f1-score   support

           0     0.8378    0.8751    0.8561      1033
           1     0.6067    0.5321    0.5670       374

    accuracy                         0.7839      1407
   macro avg     0.7223    0.7036    0.7115      1407
weighted avg     0.7764    0.7839    0.7792      1407



🔍 해석

- 이탈 고객(1) 의 precision은 60.7%, recall은 53.2%
- XGBoost보다 소폭 개선된 정밀도와 재현율 → 예측된 이탈 중 60%는 실제 이탈이며, 실제 이탈 중 절반 정도를 탐지

- 비이탈 고객(0) 은 여전히 안정적인 예측 성능 유지
- 전체적으로 로지스틱 회귀보다 약간 향상, XGBoost와 유사한 성능 → Recall이 조금 더 높은 편이라 이탈 탐지 측면에서는 유리

---

### 📌 2-5. 딥러닝 – DNN (Deep Neural Network)

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report

# 모델 구성
dnn_model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

# 컴파일
dnn_model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# 학습
dnn_model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0)

# 예측
y_pred_dnn = (dnn_model.predict(X_test) > 0.5).astype(int)

# 결과 확인
print(" [DNN]")
print(classification_report(y_test, y_pred_dnn, digits=4))

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 552us/step
 [DNN]
              precision    recall  f1-score   support

           0     0.8593    0.8161    0.8371      1033
           1     0.5540    0.6310    0.5900       374

    accuracy                         0.7669      1407
   macro avg     0.7067    0.7235    0.7136      1407
weighted avg     0.7782    0.7669    0.7714      1407



🔍 해석
- 이탈 고객(1) 의 recall은 63.1%로, 다른 트리 기반 모델보다 다소 높음
- 하지만 precision은 55.4%로, 예측된 이탈 중 실제 이탈은 절반 수준
- F1-score는 0.59로 전체 평균보다 낮지만, 이탈 탐지 비중을 더 높게 두는 분석에서는 중요한 역할 가능
- 전체 정확도는 76.7%로 평균 이상, LightGBM과 유사한 성능을 보임


---

## 📊 3. 모델 평가

In [26]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import pandas as pd

# 모델 이름과 예측값 리스트
model_names = ["Logistic Regression", "Random Forest", "XGBoost", "LightGBM", "DNN"]
y_preds = [y_pred_lr, y_pred_rf, y_pred_xgb, y_pred_lgb, y_pred_dnn]

# 결과 저장
metrics = {
    "Model": [],
    "Accuracy": [],
    "Precision": [],
    "Recall": [],
    "F1 Score": []
}

# 계산
for name, y_pred in zip(model_names, y_preds):
    metrics["Model"].append(name)
    metrics["Accuracy"].append(accuracy_score(y_test, y_pred))
    metrics["Precision"].append(precision_score(y_test, y_pred))
    metrics["Recall"].append(recall_score(y_test, y_pred))
    metrics["F1 Score"].append(f1_score(y_test, y_pred))

# 데이터프레임으로 정리
df_metrics = pd.DataFrame(metrics).round(4)
display(df_metrics)

,Model,Accuracy,Precision,Recall,F1 Score
0,Logistic Regression,0.8024,0.6446,0.5722,0.6062
1,Random Forest,0.7868,0.6201,0.5107,0.5601
2,XGBoost,0.7669,0.5676,0.5160,0.5406
3,LightGBM,0.7839,0.6067,0.5321,0.5670
4,DNN,0.7669,0.5540,0.6310,0.5900


🔍 해석

Logistic Regression
- 가장 높은 Accuracy와 F1 Score 기록. Precision과 Recall 간 균형도 좋고, 기본 모델로서 안정적인 성능을 보여줌

Random Forest / XGBoost / LightGBM
- 성능은 고만고만하지만, XGBoost는 Recall이 살짝 더 높고 Precision이 낮아 실제 이탈 고객을 더 놓치지 않으려는 경향이 있음

DNN (딥러닝)
- Recall(0.6310)이 가장 높아 이탈한 고객을 가장 잘 맞춤. 하지만 Precision이 낮아 false positive가 다소 발생 → 마케팅 입장에서는 "실제 이탈할 사람"을 잘 포착하는 점에서 의미 있음